In [1]:
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
model = tf.keras.models.load_model("saved_text_classifer.h5")

with open("./encoder.pkl", "rb") as infile:
    encoder = pickle.load(infile)

In [3]:
df_test = pd.read_csv("./names.csv")
enc_space = len(encoder.items())

In [4]:
ip = df_test[['Person_Lname', 'Person_Fname']]
op = df_test[['full_name', 'first_name', 'gen_label', 'missing']]

X_table = ip.fillna("0")
y = op.fillna(0).values

In [5]:
input_string = [i[0].upper()+"\t"+i[1].upper() for i in X_table.values.tolist()]

In [6]:
def encode_char(c):
    vec = [0]*enc_space
    vec[encoder[c]] = 1
    return vec

def encode_name(n):
    return np.array([encode_char(i) for i in n])

In [7]:
new_model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(enc_space, batch_size=1)),
    tf.keras.layers.Dense(enc_space, activation='relu'),
    tf.keras.layers.Dense(4, activation='softmax')]
)

In [8]:
input_vector_unpadded = [encode_name(i) for i in input_string]
X = pad_sequences(input_vector_unpadded, padding='post', truncating='post', maxlen=40)

In [9]:
X = X.astype('float32')

In [10]:
new_model.build(input_shape=(1, X.shape[1], X.shape[2]))
old_weights = model.get_weights()
new_model.set_weights(old_weights)
new_model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.SGD(1e-2),
              metrics=['mean_squared_error', 'accuracy'])

In [24]:
model.predict_classes(X, batch_size=X.shape[0])

array([0, 0, 0, ..., 0, 0, 0])